In [1]:
from library.plots import plot_heatmap, plot_surface
import pandas as pd
import torch as t
import time, datetime
import plotly.graph_objects as go
import torch.nn.functional as F


# profile the code with cProfile
import cProfile
import pstats

from tqdm import tqdm
from copy import deepcopy

template = "plotly_dark"

In [2]:
def time_now():
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Edge

In [14]:
edges_slits = [[-50, 10]]  # 1 slit
barrier_points = []
gravity_nodes_x = t.tensor([10])

# 1 Slit


In [15]:
edges_slits = [[-20, 20]]  # 1 slit
barrier_points = [[-35, 35]]
# barrier_points = [[-10, 10]]
gravity_nodes_x = t.tensor(edges_slits).reshape(1, -1).squeeze()

# 2 Slits


In [3]:
edges_slits = [[-5000, 0]]  # two slits
barrier_points = [[-5000, i] for i in range(1, 3)]
# gravity_nodes_x = t.tensor(edges_slits + barrier_points).reshape(1, -1).squeeze().sort()[0]
gravity_nodes_x = t.tensor(edges_slits).reshape(1, -1).squeeze().sort()[0]

# Simulation

In [38]:
device = t.device("cuda") if t.cuda.is_available() else t.device("cpu")

n_slits = len(edges_slits)
n_particles = 1_000  # <3 3080Ti
barrier_y = 0
dt = 0.001
v_y = 30  # FIX
x = 100
y = 3000  # 10000
type_trig = 'cos'
# G = -0.6626  # why is this negative?
G = 1
mass = 1  # FIX
wavelength = 0.6238
wavelength = 1
power = 1
inner_power = 1
round_x = 0
n_bins = 500  # always odd numbers
PLOT_3D = True
PLOT_Z_DISTRIBUTION = False
PLOT_DENSITY = False
normalize = True
moving_wave = True
#TODO: fix this threshold
y_threshold = 0.1 / dt

p_x = t.Tensor()
field_x_min = -x
field_x_max = x

for slit in edges_slits:
    p_x = t.cat(
        (
            p_x,
            t.linspace(
                max(slit[0], field_x_min), min(slit[1], field_x_max), n_particles
            ),
        ),
        0,
    )
    # remove 5% of the particles that would be superpositioned with edges
    # p_x = p_x[len(p_x) // 20 : -len(p_x) // 20]
p_y = t.zeros_like(p_x)
p_xy = t.stack((p_x, p_y), 1).to(device)
# Let's try to keep the trail on the cpu
trail_xy = t.clone(p_xy.unsqueeze(0))
v_xy = t.zeros_like(p_xy).to(device) + t.tensor((0, v_y)).to(device)

p_mass = t.ones_like(p_y).to(device)
o_y = t.zeros_like(gravity_nodes_x)
o_xy = t.stack((gravity_nodes_x, o_y), 1).to(device)

# Stupid work around, code is becoming cluttered
dist = t.Tensor().to(device)
hist = t.histc(trail_xy[0][:, 0].to("cpu"), bins=n_bins, min=-x, max=x)
dist = t.cat((dist, hist.unsqueeze(0).to(device)), 0)

t_ = 0
for i in tqdm(range(y)):
    if i <= y_threshold:
        dt = 10
    dt_ = t.ones_like(p_y).unsqueeze(1).to(device) * dt
    a_xy = t.zeros_like(p_xy).to(device)
    for o in o_xy:
        d_xy = p_xy - o
        r = t.sqrt(t.sum(d_xy**2, axis=1))
        r_wavelength = r**inner_power * wavelength
        force = G * mass / r**2
        wave_force = r_wavelength - v_y * t_ * moving_wave
        if type_trig == "sin":
            force *= t.sin(wave_force) ** power
        elif type_trig == "cos":
            force *= t.cos(wave_force) ** power
        elif type_trig == "tan":
            force *= t.tan(wave_force) ** power
        elif type_trig == 'test':
            force *= t.cos(t.pi * r ** 2 / wavelength)**2
        a_xy += force.unsqueeze(1) * d_xy / t.stack([r, r], axis=1)
    v_xy += a_xy * dt_
    # calculate the velocity of each particle
    v = t.sqrt(t.sum(v_xy**2, axis=1))
    # clip the velocity to the max velocity which is v_y
    v = t.clip(v, 0, v_y)
    # normalize the velocity vector
    v_xy = v_xy / t.stack([v, v], axis=1)
    # multiply the velocity vector by the max velocity
    v_xy *= v_y

    dt_ = (1 / v_xy)[:, 1].unsqueeze(1)
    t.abs_(dt_)
    p_xy += v_xy * dt_
    if round_x > 0:
        p_xy = p_xy.to("cpu").round(decimals=round_x).to(device)
    trail_xy = t.cat((trail_xy, p_xy.unsqueeze(0)), 0)
    hist = t.histc(trail_xy[i][:, 0].to("cpu"), bins=n_bins, min=-x, max=x).to(device)
    dist = t.cat((dist, hist.unsqueeze(0)), 0)
    t_ += dt

dist = pd.DataFrame(dist.cpu().numpy())
dist = dist.div(dist.max().max())
if round_x > 0:
    dist = dist.round(round_x)

params = dict(
    type_trig=type_trig,
    G=G,
    mass=mass,
    wavelength=wavelength,
    power=power,
    inner_power=inner_power,
    moving_wave=moving_wave,
    edges_slits=edges_slits,
    barrier_points=barrier_points,
    n_particles=n_particles,
    barrier_y=barrier_y,
    dt=dt,
    v_y=v_y,
    x=x,
    y=y,
    round_x=round_x,
    normalize=normalize,
)

100%|██████████| 3000/3000 [00:03<00:00, 899.45it/s] 


In [39]:
if PLOT_3D:
    print("Creating surface plot")
    plot_surface(dist, gravity_nodes_x, o_y, template, params, show=True)

if PLOT_Z_DISTRIBUTION:
    print("Creating heatmap plot")
    plot_heatmap(dist, gravity_nodes_x, o_y, template, params)

Creating surface plot


In [104]:
# Initialize the grid for plotting the gravitational field
x_range = (-50, 50)
y_range = (0, 50)  # Reduced for better visibility in the plot
step = 5  # Reduced for better resolution
x_points = t.arange(x_range[0], x_range[1], step=step, device=device)
y_points = t.arange(y_range[0], y_range[1], step=step, device=device)
xx, yy = t.meshgrid(x_points, y_points)
grid_xy = t.stack((xx.flatten(), yy.flatten()), dim=1)

# Initialize z_values to represent a flat plane initially
z_values = t.zeros((len(y_points), len(x_points)), device=device)

# Calculate the gravitational force (and hence the 'displacement') at each grid point
for o in o_xy:
    d_xy = grid_xy - o
    r = t.sqrt((d_xy**2).sum(dim=1))
    F_magnitude = G * mass / (r**2 + 1e-9)
    displacement = F_magnitude.view(len(y_points), len(x_points))
    z_values += displacement

# Assume dist_shape is the shape of 'dist', and z_values_shape is the shape of 'z_values'
dist_shape = dist.shape
z_values_shape = z_values.shape

scale_factor_y = dist.shape[0] / z_values.shape[0]
scale_factor_x = dist.shape[1] / z_values.shape[1]
z_values_interpolated = F.interpolate(
    z_values.unsqueeze(0).unsqueeze(0),
    scale_factor=(scale_factor_y, scale_factor_x),
    mode="bilinear",
    align_corners=True,
)
z_values_interpolated = z_values_interpolated.squeeze(0).squeeze(0)

In [ ]:
import torch as t
import plotly.graph_objects as go


# Function to plot the gravitational field
def plot_gravitational_field(z_values, x_range, y_range, step):
    x_points = list(range(x_range[0], x_range[1], step))
    y_points = list(range(y_range[0], y_range[1], step))
    fig = go.Figure(data=[go.Surface(z=z_values.cpu().numpy(), x=x_points, y=y_points)])
    fig.update_layout(
        title="Gravitational Field",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Gravitational Force"),
    )
    fig.show()


# Parameters
G = 2
mass = 1
x_range = (-50, 50)
y_range = (0, 50)  # Reduced for better visibility in the plot
step = 5  # Reduced for better resolution

# Set device
device = t.device("cuda") if t.cuda.is_available() else t.device("cpu")

# Gravity nodes (e.g., edges of slits)
o_xy = t.tensor([[-25, 0], [25, 0]], device=device)

# Initialize the grid and force vectors
x_points = t.arange(x_range[0], x_range[1], step=step, device=device)
y_points = t.arange(y_range[0], y_range[1], step=step, device=device)
xx, yy = t.meshgrid(x_points, y_points)
grid_xy = t.stack((xx.flatten(), yy.flatten()), dim=1)

# Initialize z_values to represent a flat plane initially
z_values = t.zeros((len(y_points), len(x_points)), device=device)

# Calculate the gravitational force (and hence the 'displacement') at each grid point
for o in o_xy:
    d_xy = grid_xy - o
    r = t.sqrt((d_xy**2).sum(dim=1))
    F_magnitude = G * mass / (r**2 + 1e-9)
    F_vector = F_magnitude.unsqueeze(1) * (d_xy / r.unsqueeze(1))

    # Calculate the 'displacement' due to this gravitational node and add it to z_values
    displacement = F_magnitude.view(len(y_points), len(x_points))
    z_values += displacement

# Plotting
plot_gravitational_field(-z_values, x_range, y_range, step)